## Custom Torch Modeling

이번 실습에서는 시계열 예측을 수행하는 class를 제작해, 전체적인 시계열 예측 파이프라인을 생성하는 것을 목표로 한다. 구조는 객체지향의 디자인 패턴을 고려하면서 만들어 보자.

### 데이터 준비

S&P500 Index (yfinance : `^GSPC`)의 일별 데이터에 대해서 가격을 다운받는다.

In [1]:
import yfinance as yf

START = '2000-01-01'
TICKER = '^GSPC'

data = yf.download(
    TICKER,
    start = START,
    progress = False,
    interval = '1d'
)

In [2]:
data['Close'].tail() # 데이터 확인

Date
2025-01-27    6012.279785
2025-01-28    6067.700195
2025-01-29    6039.310059
2025-01-30    6071.169922
2025-01-31    6040.529785
Name: Close, dtype: float64

### 1. Statistical Prediction Model

준비된 데이터에 대해서 아래의 인터페이스에 맞는 class 를 제작해 보자.

1. AR Model에 대해 1-step ahead prediction을 진행하는 class `ARForecast`
2. MA Model에 대해 1-step ahead prediction을 진행하는 class `MAForecast`
3. ARMA Model에 대해 1-step ahead prediction을 진행하는 class `ARMAForecast`

을 제작해 본다. 생성하는 클래스는 아래의 추상 클래스 `Forecast`를 상속받는다. 내부 계산에서는 `statsmodels`의 시계열 모형을 사용한다. 1-step ahead prediction을 진행하기 위해서는 학습에 사용할 window 기간을 설정하는 parameter `window`를 `fit()` method에 over-riding하여 확장한다. 

In [3]:
from abc import ABC, abstractmethod
import pandas as pd

class Forecast(ABC) :
    '''
    Abstract base class for all Forecasts
    '''
    def __init__(self, data) :
        self.data = data
    
    @abstractmethod
    def prepare_data(self) :
        """
        model fitting을 위해 데이터를 모델의 형태에 맞게 정제하는 method
        """
        pass
    
    @abstractmethod
    def fit(self) -> pd.Series :
        """
        model fitting을 진행하는 method
        :return: fitted 된 pandas.Series 형태의 적합화된 시계열 데이터
        """
        pass
    
    @abstractmethod
    def predict(self) -> pd.Series :
        '''
        fitting된 모형을 기반으로 예측값을 출력하는 method
        :return: pd.Series 형태의 예측된 데이터
        '''
        pass
    
    @abstractmethod
    def mse_score(self) -> float :
        '''
        예측 모형과 실제 값 간의 mean squared error를 계산하는 method
        :return: 
        '''
        pass

In [ ]:
# 이 부분에서 코드를 작성해 주세요
class ARForecast(Forecast) :
    pass

class MAForecast(Forecast) :
    pass

class ARMAForecast(Forecast) :
    pass

### 2. Torch Prediction Model

1번에서 사용된 인터페이스 `Forecast`를 상속하여 

1. RNN에 대해 test기간에서의 예측을 수행하는 class `RNNForecast`
2. GRU에 대해 test기간에서의 예측을 수행하는 class `GRUForecast`
3. LSTM에 대해 test기간에서의 예측을 수행하는 class `LSTMForecast`

를 생성해 본다. 여기서 Torch 기반의 예측 클래스에서는 train 기간과 test 기간을 따로 사용해야 하기 때문에, `fit()`에서는 학습 데이터를 넣고, `predict()`에서는 실습 데이터를 넣어서 예측값을 생성하도록 디자인한다. RNN 모형, GRU 모형과 LSTM모형은 강의자료에 있는 클래스를 그대로 받아서 사용해도 무방하다. (혹은 layer를 임의로 변경하여도 상관없다)

In [ ]:
# 이 부분에서 코드를 작성해 주세요
class RNNForecast(Forecast) :
    pass

class GRUForecast(Forecast) :
    pass

class LSTMForecast(Forecast) :
    pass

### 3. Generalized Forecast Module

1번과 2번에서 작성한 Forecast Module을 통합적으로 사용하기 위해 class `TimeSeriesForecast`를 생성한다. 예상되는 use case는 다음과 같다.

```{python}
# Torch Model을 사용하는 경우
forecast = TimeSeriesForecast(train_data, method = 'RNN')
forecast.fit(p = 2) # 사용할 과거 데이터의 개수
y_pred = forecast.predict(test_data)

# Statistical Model을 사용하는 경우
forecast = TimeSeriesForecast(data, method = 'AR')
forecast.fit(p = 2, window = 252) # p : 사용할 과거 데이터의 개수, window : 1기간 예측값을 생성할 때 사용할 데이터의 개수
y_pred = forecast.predict()
```

In [ ]:
# 이 부분에서 코드를 작성해 주세요